In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# plot the function x**4
x_four = lambda x: x**4

lin_space = np.linspace(-0.5, 0.5)

initial_fig = px.line(x=lin_space, y=x_four(lin_space))

initial_fig

Q1: Describe the function $y=x^4$.
*   What is its minimum? How do you compute it?\
It is $(x,y) = (0,0)$. Compute the first derivative and equate it to 0.
*   What is its curvature? To what derivative is it related?\
It is related to the acceleration (second derivative).
$$\frac{\partial^2}{\partial x^2} x^4$$
*   What happens to the curvature of the function when $x→0$, $|x|→∞$?\
The curvature goes to 0.
*   Is this curvature pathological? \
No.


## GD for large learning rates

Let $f: \mathbb{R}^n \to \mathbb{R}$ be a convex function with a unique mimimum Let $x_t$ be our current estimate of its minimum. Gradient descent updates $x_t$ by noting that the gradient gives the direction of greatest increase of the function, and thus to minimize the function we should move in the opposite direction. It uses the following update rule:

$$
    x_{t+1}=x_t-\alpha \nabla f(x_t)
$$

where $\alpha$ is the learning rate and $\nabla f(x_t)$ is the gradient.

Initialize $x=0.5$ and let $\alpha=1.999999999999999$. Write the GD algorithm.


In [3]:
# set-up GD algorithm with lr = 1.999999999999999

# initialize x = 0.5 and y
x = 0.5
y = x**4

# set n_iter and lr
n_iter = 1000
lr = 1.999999999999999


# define derivative
deriv = lambda x: 4 * x ** 3

def vanilla_gd(x, y, lr, deriv, n_iter=1000):
    """
    Simple implementation of vanilla gradient descent
    """
    # initialize buffer for results
    x_buf = np.zeros(n_iter)
    # GD algorithm
    for i in range(n_iter):
        x_buf[i] = x
        
        # compute derivative d_y
        d_y = deriv(x)

        # apply GD step
        
        x = x - lr * d_y

        # store result in buffer
    x_buf[i] = x
    return x_buf
vanilla_gd(x, y, lr, deriv)

array([ 0.5       , -0.5       ,  0.5       , -0.5       ,  0.5       ,
       -0.5       ,  0.5       , -0.5       ,  0.5       , -0.5       ,
        0.5       , -0.49999999,  0.49999997, -0.49999983,  0.49999915,
       -0.49999576,  0.49997882, -0.49989413,  0.49947077, -0.49735722,
        0.48686978, -0.4363996 ,  0.22848003,  0.13306106,  0.11421403,
        0.1022948 ,  0.09373131,  0.08714345,  0.08184933,  0.07746265,
        0.07374416,  0.07053588,  0.06772837,  0.06524294,  0.06302121,
        0.06101882,  0.05920129,  0.05754138,  0.05601722,  0.054611  ,
        0.05330804,  0.05209614,  0.05096502,  0.04990599,  0.04891162,
        0.04797552,  0.04709213,  0.04625666,  0.04546486,  0.04471303,
        0.04399789,  0.04331652,  0.04266631,  0.04204495,  0.04145034,
        0.0408806 ,  0.04033404,  0.0398091 ,  0.0393044 ,  0.03881865,
        0.03835069,  0.03789944,  0.03746394,  0.03704328,  0.03663664,
        0.03624323,  0.03586237,  0.03549339,  0.03513568,  0.03

In [4]:
def plot_vanilla_gd(x, y, lr, deriv):
    # plot results

    x_buf = vanilla_gd(x, y, lr, deriv)

    vanilla_gd_fig = go.Figure()
    vanilla_gd_fig.add_trace(initial_fig.data[0])
    vanilla_gd_fig.add_trace(go.Scatter(x=x_buf, y=x_four(x_buf), mode="lines+markers", name="vanilla_gd"))
    return vanilla_gd_fig


plot_vanilla_gd(x, y, lr, deriv)

Q2: Describe the figure.
*   What happens in high curvature regions?\
We overshoot.
*   Try to set $\alpha=2$. Is the bounce more evident?\
See below
*   How can we potentially reduce this bouncing?\
Diminishing learning rate.



In [5]:
plot_vanilla_gd(x=x, y=y, lr=2, deriv=deriv)

## GD for small learning rates

Set the learning rate to $\alpha=0.5$ and run the algorithm again.

In [6]:
# set-up GD algorithm with lr = 0.5

plot_vanilla_gd(x=x, y=y, lr=0.5, deriv=deriv)

Q3: Describe the figure.
*   Do we still have bouncing in the region of high curvature?\
No.
*   What happened in the region of low curvature?\
We move slowly.


## Momentum

Based on the above results, we want two things:
1.   To make smaller steps in regions of high curvature to dampen oscillations;
2.   To make larger steps and accelerate in regions of low curvature.

One way to do both is to guide the next steps towards the previous direction. We can achieve this via a simple tweak to the gradient descent update rule:

$$
x_{t+1}=x_t-\alpha\nabla f(x_t)+\underbrace{\beta(x_t-x_{t-1})}_{\textrm{momentum}}
$$
This idea comes from Polyak [1], and is also called the heavy ball method. Intuitively, a heavier ball will bounce less and move faster through regions of low curvature than a lighter ball due to momentum. This has two effects, related to the two goals stated:

1.   We penalize changes in direction to take smaller steps;
2.   When the direction doesn’t change we take bigger steps.

[1] Polyak, Boris T. “Some methods of speeding up the convergence of iteration methods.” USSR Computational Mathematics and Mathematical Physics 4, no. 5 (1964): 1-17.

Let’s first apply momentum in the setting where we bounced across the walls ($\alpha = 2$). We set $\beta = 0.5$.

In [8]:
# apply momentum when lr = 2., with mom = 0.5

x = 0.5
y = x**4

n_iter = 1000
lr = 2.
mom = 0.5

# x_buf = np.zeros(n_iter)
# x_buf[-1] = x
# x_buf[0] = x

def momentum_gd(x, y, lr, deriv, n_iter=1000):
    """
    Simple implementation of momentum gradient descent
    """
    # initialize buffer for results
    x_buf = np.zeros(n_iter)
    x_buf[-1] = x
    x_buf[0] = x
    
    # GD algorithm
    for i in range(n_iter):
        x_buf[i] = x
        
        
        # compute derivative d_y
        d_y = deriv(x)

        # compute momentum change
        mom_diff = x_buf[i] - x_buf[i-2]


        # apply GD step
        
        x = x - (lr * d_y) + mom * mom_diff

        # store result in buffer
    x_buf[i] = x
    
    return x_buf

def plot_momentum_gd(x, y, lr, deriv):
    # plot results

    x_buf = momentum_gd(x, y, lr, deriv)

    momentum_gd_fig = go.Figure()
    momentum_gd_fig.add_trace(initial_fig.data[0])
    momentum_gd_fig.add_trace(go.Scatter(x=x_buf, y=x_four(x_buf), mode="lines+markers", name="momentum_gd"))
    return momentum_gd_fig

plot_momentum_gd(x, y, lr, deriv)

Q4. Discussion on the figure.
*   Do we converge or diverge when $\alpha = 2$?\
Converge.
*   What is the effect of momentum in high curvature regions?\
Decrease the corrective term
*   What is the effect of momentum in low curvature regions?\
Increase the corrective term